# RA2CE feature: Damages analysis (manual)

Three types of damage curves can be introduced to this analysis:
- Huizinga
- OSdaMage 
- User defined

## An example for a manual damage function

In this notebook, we provide an example for an event-based object-oriented Huizinga analysis.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import rasterio
from ra2ce.ra2ce_handler import Ra2ceHandler #import the ra2cehandler to run ra2ce analyses

root_dir = Path("data", "damages_analysis_manual")
assert root_dir.exists(), "root_dir not found."

#### Introducing a hazardous event

To use the flood map with RA2CE, we need to fill in the **[hazard]** section in the network.ini. 

Specify the flood map name in the **hazard_map** parameter in network.ini. RA2CE expects the flood map to be located in the *hazard* folder. The **aggregate_wl** parameter in analysis.ini can be set to either 'max', 'min' or 'mean' to take the maximum, minimum or mean water depth per road segment when the exposure of the roads to a certain hazard (map) is determined.<br>

Set the right CRS for the flood map in the **hazard_crs** parameter. This CRS can be different from the origins, destinations and road network. RA2CE will reproject the network to the CRS of the flood map and will reproject the road back to the original CRS when the CRS differs.

In [ ]:
hazard_folder = root_dir / "static" / "hazard" # find the hazard folder where you locate your floo dmap
hazard_map = hazard_folder / "max_flood_depth.tif" # set the location of the hazard map

# Open the TIF file using rasterio
with rasterio.open(hazard_map) as src:
    # Read the TIF file as a numpy array
    tif_array = src.read(1)  # Change the band index (1) if necessary

plt.figure(figsize=(10, 10))
plt.imshow(tif_array, cmap='Blues')  # Change the colormap if desired
plt.colorbar(label='Pixel Values')
plt.title('Flood map')
plt.show()


#### Specifying the .ini files

**Network.ini content**
<br>
<br>
> [project]<br>
name = beira<br>
<br>
[network]<br>
**directed = False<br>
source = OSM download<br>
primary_file = None<br>
diversion_file = None<br>
file_id = rfid_c<br>
polygon = region_polygon.geojson<br>
network_type = drive<br>
road_types = motorway,motorway_link,primary,primary_link,secondary,secondary_link,tertiary,tertiary_link,residential<br>
save_gpkg = True
<br>
[origins_destinations]<br>
**origins = None<br>
destinations = None<br>
origins_names = None<br>
destinations_names = None<br>
id_name_origin_destination = None<br>
origin_count = None<br>
origin_out_fraction = None<br>
category = category**<br>
<br>
[hazard]<br>
**hazard_map = max_flood_depth.tif<br>
hazard_id = None<br>
hazard_field_name = waterdepth<br>
aggregate_wl = max<br>
hazard_crs = EPSG:32736**<br>
<br>
*[cleanup]<br>
snapping_threshold = None<br>
segmentation_length = 100<br>
merge_lines = True<br>
merge_on_id = False<br>
cut_at_intersections = False<br>*

We now need to update our analysis initialisation files using the preferred OD-analysis (there are multiple). We will consider the **damages** analysis. With the **aggregate_wl** parameter, the user can choose which type of aggregation of the water level on the road segment (max, mean, min) the analysis should consider. For the damages analysis, the **aggregate_wl=mean** makes sense. 

The **damage_curve** defines the damage curve type. ‘HZ’ to use the Huizinga damage function, ‘OSD’ to use the OSdaMage functions, and ‘MAN’ to use damage functions from manually inserted files. The **event_type** defines the type of the hazardous event, which is either 'event', or 'return_period'. The former is an one-time event, while the later will be applied for the events witha probability of occurance.

**[analysis1]**
<br>
<br>
name = Manual_damages_event<br>
analysis = damages<br>
event_type = event<br>
damage_curve = MAN<br>
save_csv = True<br>
save_gpkg =True<br>

Set the paths to the initialization files and check if the files exist.

In [ ]:
from ra2ce.ra2ce_handler import Ra2ceHandler
from ra2ce.network.network_config_data.network_config_data_reader import NetworkConfigDataReader
from ra2ce.analysis.analysis_config_data.analysis_config_data_reader import AnalysisConfigDataReader

# Load network data.
_network_config_data = NetworkConfigDataReader().read(root_dir.joinpath("network.ini"))

# Load analysis data.
_analysis_config_data = AnalysisConfigDataReader().read(root_dir.joinpath("analysis.ini"))

_analysis_config_data.input_path = root_dir.joinpath("input_data")

Run RA2CE.

In [ ]:
handler = Ra2ceHandler.from_config(_network_config_data, _analysis_config_data)

handler.configure()
handler.run_analysis()

#### Visualising the results

In [ ]:
analysis_output_path = root_dir / "output" / "damages"
damage_gdf = gpd.read_file(analysis_output_path / 'Manual_damages_event_segmented.gpkg')
damage_gdf.head() #show the origins